In [35]:
import json
from pprint import pprint
import pandas as pd
import os
import re
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math

import pickle
import random
from pathlib import Path
import catboost as catb
from catboost import CatBoostClassifier

import json
from pathlib import Path
import catboost as catb
from catboost import CatBoostClassifier
import math


In [36]:
DATA_ROOT = Path('C:/Users/druzh/Desktop/data_scientist/SARAWAN/' )
MODELS_PATH = Path('C:/Users/druzh/Desktop/data_scientist/SARAWAN/models/')
DATASET_PATH = DATA_ROOT / 'df_for_predict.csv'
MODEL_FILE_PATH = MODELS_PATH / 'model.pkl'

In [37]:
model = pickle.load(open(MODEL_FILE_PATH, 'rb'))

In [38]:
# Получаем список файлов в папки
ds = pd.read_json('2606.json')

In [39]:
# ds.drop(['product_name_lower_sorted', 'product_name_lower', ], axis=1, inplace=True)
ds.name_store.value_counts()

okey              29856
globus            28768
auchan            25943
lenta             22552
metro             17902
victoria           8220
dixy               3291
magnit_express     3246
fixpricefd         2238
Name: name_store, dtype: int64

In [40]:
# Создание новых признаков, предобработка данных
def preprocess_data(df):
    NUM_FEATURE_NAMES = ['product_price','total_weight_kg']
    CAT_FEATURE_NAMES = ['first_prod_name','second_prod_name']
    res = df['product_name'].str.split(' ',expand=True).apply(lambda x: x.str.strip()).loc[:,:1]
    res.columns = CAT_FEATURE_NAMES
    res2 = df[NUM_FEATURE_NAMES]    
    df = res2.join(res)
    df.loc[df['second_prod_name'].isna()] = '-'
    return df

In [41]:
def give_me_dataset(data):
    
    # regex for all nums and len(word)>3 -> new column product_group_name
    df = data.copy()
    
    # Приведение цены товара к цене за 1 кг

#     splitted_weight = pd.DataFrame(map(lambda x: x.split(' '), 
#                                 df[df['product_price_type'] == 'per_kilo']['product_human_volume']))
#     splitted_weight = splitted_weight.set_index(df[df['product_price_type'] == 'per_kilo']['product_human_volume'].index)
#     splitted_weight[0] = splitted_weight[0].str.replace(',', '.')

#     for item in splitted_weight.index:
#         if splitted_weight[1][item] == 'кг':
#             df['product_price'][item] = round(df['product_price'][item] 
#                                                                    * 1/float(splitted_weight[0][item]), 2)
#             df['product_human_volume'][item] = '1 кг'
            
#         elif splitted_weight[1][item] == 'г':
#             df['product_price'][item] = round(df['product_price'][item] 
#                                                                    * 1000/float(splitted_weight[0][item]), 2)
#             df['product_human_volume'][item] = '1 кг'


    # Приведение цены товара к цене за 1 кг

    df.loc[(df['product_price_type'] == 'per_kilo'), 'product_price'] = df['product_unit_price']
    df.loc[(df['product_price_type'] == 'per_kilo'), 'product_human_volume'] = '1 кг'

    # Создание колонки для группировки продуктов
    re1 = r"\d+" # nums
    re2 = '\w{3,30}' # words
    list_of_names = [sorted(re.compile("(%s|%s)" % (re1, re2)).findall(str(i.lower()))) for i in df.product_name]
    list_of_weight = [re.findall(re1, i) for i in df.product_human_volume]
    small_name_col = []
    for num, w_or_d in enumerate(list_of_names):
#         if len(w_or_d) <= 3 & (re.search(re1, str(w_or_d)) is None):
        if (re.search(re1, str(w_or_d)) is None):
            weight = list_of_weight[num]
            small_name  = str(weight[0]) + ' ' + ' '.join(w_or_d)
        else:
            small_name = ' '.join(w_or_d)
        small_name_col.append(small_name)
    
    df.insert(loc=0, column='product_group_name', value=small_name_col)    
        
    # delete duplicates in each shop and make clear_dataset   
    magnit_express_df = df[df['name_store'] == 'magnit_express']
    magnit_express_df.drop_duplicates(subset='product_group_name', inplace=True)
    magnit_express_df.loc[magnit_express_df['name_store'] == 'magnit_express', 'name_store'] = 'Магнит'
    
    lenta_df = df[df['name_store'] == 'lenta']
    lenta_df.drop_duplicates(subset='product_group_name', inplace=True)
    lenta_df.loc[lenta_df['name_store'] == 'lenta', 'name_store'] = 'Лента'
    
    fixprice_df = df[df['name_store'] == 'fixpricefd']
    fixprice_df.drop_duplicates(subset='product_group_name', inplace=True)
    fixprice_df.loc[fixprice_df['name_store'] == 'fixpricefd', 'name_store'] = 'Фикспрайс'
    
    metro_df = df[df['name_store'] == 'metro']
    metro_df.drop_duplicates(subset='product_group_name', inplace=True)
    metro_df.loc[metro_df['name_store'] == 'metro', 'name_store'] = 'Метро'
    
    victoria_df = df[df['name_store'] == 'victoria']
    victoria_df.drop_duplicates(subset='product_group_name', inplace=True)
    victoria_df.loc[victoria_df['name_store'] == 'victoria', 'name_store'] = 'Виктория'
    
    auchan_df = df[df['name_store'] == 'auchan']
    auchan_df.drop_duplicates(subset='product_group_name', inplace=True)
    auchan_df.loc[auchan_df['name_store'] == 'auchan', 'name_store'] = 'Ашан'
    
    okey_df = df[df['name_store'] == 'okey']
    okey_df.drop_duplicates(subset='product_group_name', inplace=True)
    okey_df.loc[okey_df['name_store'] == 'okey', 'name_store'] = 'Окей'
    
    globus_df = df[df['name_store'] == 'globus']
    globus_df.drop_duplicates(subset='product_group_name', inplace=True)
    globus_df.loc[globus_df['name_store'] == 'globus', 'name_store'] = 'Глобус'
    
    
    globus_df = df[df['name_store'] == 'dixy']
    globus_df.drop_duplicates(subset='product_group_name', inplace=True)
    globus_df.loc[globus_df['name_store'] == 'dixy', 'name_store'] = 'Дикси'
    
    
    globus_df = df[df['name_store'] == 'verniy_fd']
    globus_df.drop_duplicates(subset='product_group_name', inplace=True)
    globus_df.loc[globus_df['name_store'] == 'verniy_fd', 'name_store'] = 'Верный'
    
    clear_dataset = pd.concat([magnit_express_df,lenta_df,metro_df,victoria_df,auchan_df,okey_df,globus_df, fixprice_df])
    
    # delete inedible goods
    clear_dataset.drop(clear_dataset[clear_dataset.product_human_volume == '50 м'].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.product_human_volume.str.contains('см')|
#                                      clear_dataset.product_human_volume.str.contains('лист')|
#                                      clear_dataset.product_human_volume.str.contains('Вт')|
#                                      clear_dataset.product_human_volume.str.contains('Гб')|
#                                      clear_dataset.product_human_volume.str.contains('пак')|
#                                      clear_dataset.product_human_volume.str.contains('уп')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.main_category_name.str.contains('Электрон')|
#                                      clear_dataset.main_category_name.str.contains('емонт')|
# #                                      clear_dataset.main_category_name.str.contains('Канцелярия')|
#                                      clear_dataset.main_category_name.str.contains('гигиена')|
#                                      clear_dataset.main_category_name.str.contains('Товары для дома')|
#                                      clear_dataset.main_category_name.str.contains('обувь')|
#                                      clear_dataset.main_category_name.str.contains('химия')|
#                                      clear_dataset.main_category_name.str.contains('Авто')|
#                                      clear_dataset.main_category_name.str.contains('Дача')|
#                                      clear_dataset.main_category_name.str.contains('екстиль')|
#                                      clear_dataset.main_category_name.str.contains('книги')|
#                                      clear_dataset.main_category_name.str.contains('порт')|
#                                      clear_dataset.main_category_name.str.contains('-10% со СберПрайм')|
#                                      clear_dataset.main_category_name.str.contains('Спецпредложения')|
#                                      clear_dataset.main_category_name.str.contains('Рекомендац')|
#                                      clear_dataset.main_category_name.str.contains('Алкоголь')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.product_name.str.contains('play')|
#                                      clear_dataset.product_name.str.contains('Play')|
#                                      clear_dataset.product_name.str.contains('игурка')|
#                                      clear_dataset.product_name.str.contains('пластилин')|
#                                      clear_dataset.product_name.str.contains('песок')|
#                                      clear_dataset.product_name.str.contains('онструктор')|
#                                      clear_dataset.product_name.str.contains('Пенал')|
#                                      clear_dataset.product_name.str.contains('Краски')|
#                                      clear_dataset.product_name.str.contains('Салфетки')|
#                                      clear_dataset.product_name.str.contains('Ванночка')].index, inplace=True)
    
#     clear_dataset.drop(clear_dataset[clear_dataset.category_name.str.contains('увениры')|
#                                      clear_dataset.category_name.str.contains('пятновыводители')|
#                                      clear_dataset.category_name.str.contains('пароварки')|
#                                      clear_dataset.category_name.str.contains('оловные уборы')|
#                                      clear_dataset.category_name.str.contains('езодорант')|
#                                      clear_dataset.category_name.str.contains('ирлянд')|
#                                      clear_dataset.category_name.str.contains('ксессуар')|
#                                      clear_dataset.category_name.str.contains('дом')|
#                                      clear_dataset.category_name.str.contains('игиена')|
#                                      clear_dataset.category_name.str.contains('Авто')|
#                                      clear_dataset.category_name.str.contains('игры')|
#                                      clear_dataset.category_name.str.contains('музыка')|
#                                      clear_dataset.category_name.str.contains('уход')|
#                                      clear_dataset.category_name.str.contains('Зубн')|
# #                                      clear_dataset.category_name.str.contains('Игруш')|
#                                      clear_dataset.category_name.str.contains('Купание')|
#                                      clear_dataset.category_name.str.contains('переноск')|
#                                      clear_dataset.category_name.str.contains('антисептик')|
#                                      clear_dataset.category_name.str.contains('Наполнител')|
#                                      clear_dataset.category_name.str.contains('Носки')|
#                                      clear_dataset.category_name.str.contains('посуда')|
#                                      clear_dataset.category_name.str.contains('рабочего')|
#                                      clear_dataset.category_name.str.contains('Ортопед')|
#                                      clear_dataset.category_name.str.contains('Пазлы')|
#                                      clear_dataset.category_name.str.contains('упаковка')|
#                                      clear_dataset.category_name.str.contains('Презервативы')|
#                                      clear_dataset.category_name.str.contains('Присыпка')|
#                                      clear_dataset.category_name.str.contains('Прогулка')|
#                                      clear_dataset.category_name.str.contains('Пустышки')|
#                                      clear_dataset.category_name.str.contains('декор')|
# #                                      clear_dataset.category_name.str.contains('салфетки')|
# #                                      clear_dataset.category_name.str.contains('Шампуни')|
#                                      clear_dataset.category_name.str.contains('Школьные')|
#                                      clear_dataset.category_name.str.contains('Шарики')|
#                                      clear_dataset.category_name.str.contains('игруш')|
#                                      clear_dataset.category_name.str.contains('Подгузники')].index, inplace=True)
    
    
    volume_list = clear_dataset['product_human_volume'].to_list().copy()
    volume_list = [i.replace(',', '.') for i in volume_list]
    volume_list = [i.split() for i in volume_list]
    new_list = []
    re3 = r"\d+[ ]\w{1,2}"
    re4 = r"\d+"
    for num, i in enumerate(volume_list):
        if len(i) == 4:
            volume_list[num] = str(float(i[0]) * float(i[3])) + ' ' + i[1]
        if len(i) == 2 and i[1] == 'шт':
            try:
                volume_list[num] = re.compile("(%s|%s)" % (re3, re4)).findall(str(new_df.product_name.to_list()[num]))[0].split()
            except:
                volume_list[num] = '1 кг'
            
    for num, i in enumerate(volume_list):
        if len(i) == 2:
            if i[1] == 'г' or i[1] == 'мл':
                new_list.append(round(float(i[0]) / 1000, 2))
            elif i[1] == 'мг':
                new_list.append(round(float(i[0]) / 1000000, 2))
            elif i[1] == 'л':
                new_list.append(round(float(i[0]), 2))
            elif i[1] == 'шт':
                new_list.append(1)
            elif i[1] == 'кг':
                new_list.append(round(float(i[0])))
            else:
                new_list.append(1)
        else:
            new_list.append(1)
    
    for num, i in enumerate(new_list):
        if float(i) > 30:
            new_list[num] = float(i) / 1000
            
    discount = [round((1 - (i / j)) * 100) for i, j in zip(clear_dataset.product_unit_price.to_list(), clear_dataset.product_original_unit_price.to_list())]
    
    clear_dataset.drop(['product_discount'], axis=1, inplace=True)
    clear_dataset.insert(loc=20, column='product_discount', value=discount)   
    clear_dataset.insert(loc=0, column='total_weight_kg', value=new_list)
    
    short_ds = preprocess_data(clear_dataset)
    clear_dataset['main_category_name'] = model.predict(short_ds)
    
    for i in range(0, math.ceil(clear_dataset.shape[0]/10000)):
        rg1 = int(f'{i}0000')
        rg2 = int(f'{i+1}0000')
        with open(f'data/sber_{i}.json', 'w', encoding='utf-8') as file:
            clear_dataset[rg1:rg2].to_json(file, force_ascii=False)
    
    
    
    return clear_dataset

In [42]:
%%time
new_df = give_me_dataset(ds)

<ipython-input-41-3be47eca3298>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  magnit_express_df.drop_duplicates(subset='product_group_name', inplace=True)
<ipython-input-41-3be47eca3298>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lenta_df.drop_duplicates(subset='product_group_name', inplace=True)
<ipython-input-41-3be47eca3298>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fixprice_df.drop_duplicates(subset='product_grou

Wall time: 4min 34s


In [43]:
new_df.head(50)

,total_weight_kg,product_group_name,_id,name_store,main_category_name,main_category_img_url,category_name,category_img_url,category_permalink,product_name,...,product_price_type,product_stock,product_stock_rate,product_max_stock_rate,product_discounted,product_discount,product_unit_price,product_description,product_description_original,product_properties
24,0.40,400 ароматный коломенское кориандром нарезанны...,{'$oid': '62b829229f7c1a6b847ba701'},Магнит,Замороженные продукты,{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Хлеб Коломенское Ароматный с солодом и корианд...,...,per_item,4,0,0,False,0,60.0,Темный ароматный батон с натуральным солодом и...,Темный ароматный батон с натуральным солодом и...,{'product_ingredients': {'Состав': 'Мука пшени...
26,0.27,270 mission круглый лаваш оригинальный пшеничный,{'$oid': '62b829229f7c1a6b847ba703'},Магнит,"Молочные продукты, яйца, растительные продукты",{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Лаваш Mission оригинальный круглый пшеничный 2...,...,per_item,9,0,0,True,12,67.0,ООО «Мишн Фудс Ступино» - часть международной ...,ООО «Мишн Фудс Ступино» - часть международной ...,{'product_ingredients': {'Состав': 'Мука пшени...
28,0.24,4 60 bimbo булочки кунжутом пшеничные,{'$oid': '62b829229f7c1a6b847ba705'},Магнит,Готовая еда,{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Булочки Bimbo пшеничные с кунжутом 60 г х 4 шт,...,per_item,19,0,0,False,0,92.0,,,{'product_ingredients': {'Состав': 'Мука пшени...
29,0.27,3 90 mission деревенский лаваш тонкий,{'$oid': '62b829229f7c1a6b847ba706'},Магнит,"Молочные продукты, яйца, растительные продукты",{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Лаваш Mission Деревенский тонкий 90 г х 3 шт,...,per_item,6,0,0,False,0,80.0,<р>Лаваш MISSION изготовлен по оригинальной ре...,<р>Лаваш MISSION изготовлен по оригинальной ре...,{'product_ingredients': {'Состав': 'Мука пшени...
31,0.25,250 deli mission лепешки пшеничные тортильи,{'$oid': '62b829229f7c1a6b847ba708'},Магнит,"Молочные продукты, яйца, растительные продукты",{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Лепешки Mission Deli Тортильи пшеничные 250 г,...,per_item,8,0,0,False,0,83.0,Попробуйте наш самый популярный продукт - ориг...,Попробуйте наш самый популярный продукт - ориг...,{'product_ingredients': {'Состав': 'Мука пшени...
32,0.30,300 даниловский зерновой коломенское пшеничный...,{'$oid': '62b829229f7c1a6b847ba709'},Магнит,Готовая еда,{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Хлеб Коломенское Даниловский зерновой пшеничны...,...,per_item,8,0,0,True,29,50.0,Авторский хлеб от лучшего пекаря России Дмитри...,Авторский хлеб от лучшего пекаря России Дмитри...,{'product_ingredients': {'Состав': 'Мука пшени...
33,0.35,350 армянский ватутинки лаваш пшеничный тонкий...,{'$oid': '62b829229f7c1a6b847ba70a'},Магнит,"Молочные продукты, яйца, растительные продукты",{'url': 'https://storage.yandexcloud.net/sberm...,"Хлеб, выпечка",{'url': 'https://storage.yandexcloud.net/sberm...,na-dachu-s-bonusami-sber/khlieb-vypiechka-new,Лаваш Ватутинки Хлеб Армянский пшеничный тонки...,...,per_item,9,0,0,False,0,58.0,Хлеб является неизменной частью наших повседне...,Хлеб является неизменной частью наших повседне...,{'product_ingredients': {'Состав': 'Мука высше...
34,0.30,300 зерновой край нарезке пшеничный ржано ржан...,{'$oid': '6